In [9]:
import MultiNEAT as NEAT
from MultiNEAT import GetGenomeList, ZipFitness, EvaluateGenomeList_Serial, EvaluateGenomeList_Parallel

kw={
    'gym_name': 'FastsimSimpleNavigation-v0',
    'env_params': {"still_limit": 10, 'reward_kind': "continuous"},

    'nb_input': 5, # number of NN inputs
    'nb_output': 2, # number of NN outputs
    'nb_layers': 2, # number of layers
    'nb_neurons_per_layer': 10, # number of neurons per layer
    'max_action': 2,
    'episode_nb_step': 1000, 
    'episode_reward_kind': 'final', 
    'episode_bd': 'robot_pos', 
    'episode_bd_slice': (0,2,None), 
    'episode_bd_kind': 'final', 
    'episode_log': {'collision': 'cumul', 
                    'dist_obj': 'final', 
                    'exit_reached': 'final', 
                    'robot_pos': 'final'},
    'dim_grid': [100, 100],
    'grid_min_v': [0,0],
    'grid_max_v': [600,600],
    'goal': [60,60],
    'watch_max': 'dist_obj',
    'min_value': -30, # min genotype value
    'max_value': 30, # max genotype value
    'min_strategy': 0.5, # min value for the mutation
    'max_strategy': 3, # max value for the mutation
    'nb_gen': 25, # number of generations
    'mu': 100, # population size
    'lambda': 200, # number of individuals generated
    'nov_k': 15, # k parameter of novelty search
    'nov_add_strategy': "random", # archive addition strategy (either 'random' or 'novel')
    'nov_lambda': 6, # number of individuals added to the archive
    'selection' : "blablabla"
}

In [10]:
def eval_nn(genome, resdir = "res", render=False, dump=False, name="", **kwargs):
    """ Evaluation of a neural network. Returns the fitness, the behavior descriptor and a log of what happened
        Consider using dump=True to generate log files. These files are put in the resdir directory.
    """
    env = gym.make(kwargs['gym_name'], **kwargs['env_params'])
    nbstep=kwargs["episode_nb_step"]
    
    net = NEAT.NeuralNetwork()
    genome.BuildPhenotype(net)
    
    observation = env.reset()
    observation, reward, done, info = env.step([0]*kwargs["nb_output"])

    #print("First observation: "+str(observation)+" first pos: "+str(env.get_robot_pos()))
    if (dump):
        f={}
        for k in info.keys():
            fn=resdir+"/traj_"+k+"_"+name+".log"
            if (os.path.exists(fn)):
                cpt=1
                fn=resdir+"/traj_"+k+"_"+name+"_%d.log"%(cpt)
                while (os.path.exists(fn)):
                    cpt+=1
                    fn=resdir+"/traj_"+k+"_"+name+"_%d.log"%(cpt)
            f[k]=open(fn,"w")

    action_scale_factor = env.action_space.high
    episode_reward=0
    episode_bd=None
    episode_log={}
    for t in range(nbstep):
        if render:
            env.render()
        
        net.Input(observation)
        net.Activate()
        action = net.Output()
        #print("Observation: "+str(observation)+" Action: "+str(action))
        observation, reward, done, info = env.step(action) 
        if (kwargs["episode_reward_kind"] == "cumul"):
            episode_reward+=reward

        for k in kwargs["episode_log"].keys():
            if (kwargs["episode_log"][k] == "cumul"):
                if (k not in episode_log.keys()):
                    episode_log[k] = info[k]
                else:
                    episode_log[k] += info[k]
        if(dump):
            for k in f.keys():
                if (isinstance(info[k], list) or isinstance(info[k], tuple)):
                    data=" ".join(map(str,info[k]))
                else:
                    data=str(info[k])
                f[k].write(data+"\n")
        if(done):
            break
    if (dump):
        for k in f.keys():
            f[k].close()

    if (kwargs["episode_reward_kind"] == "final"):
        episode_reward=reward
        
    if (kwargs["episode_bd_kind"] == "final"):
        episode_bd=info[kwargs["episode_bd"]][slice(*kwargs["episode_bd_slice"])]
        
    for k in kwargs["episode_log"].keys():
        if (kwargs["episode_log"][k] == "final"):
            episode_log[k] = info[k]
    
    if(episode_log["exit_reached"]==1.0):
            print("Target REACHED ! ")
    #print("End of eval,  total_dist=%f"%(episode_reward))
    episode_reward = -episode_reward
    episode_reward = 10000-episode_reward
    #print("REWARD : ",episode_reward)
    return episode_reward, Behavior(episode_bd[0],episode_bd[1]) , episode_log

In [25]:
import math
import time

import MultiNEAT as NEAT
import gym, gym_fastsim

import progressbar as pbar
import numpy as np

class Behavior:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance_to(self, other):
        return np.sqrt((other.x - self.x)**2 + (other.y - self.y)**2)


ns_on = 1

ns_K = 15
ns_recompute_sparseness_each = 20
ns_P_min = 10.0
ns_dynamic_Pmin = True
ns_Pmin_min = 1.0
ns_no_archiving_stagnation_threshold = 150
ns_Pmin_lowering_multiplier = 0.9
ns_Pmin_raising_multiplier = 1.1
ns_quick_archiving_min_evals = 8


max_evaluations = 30000

screen_size_x, screen_size_y = 600, 600
max_timesteps = 1200



params = NEAT.Parameters()
params.PopulationSize = 200
params.DynamicCompatibility = True
params.AllowClones = False
params.AllowLoops = True
params.CompatTreshold = 5.0
params.CompatTresholdModifier = 0.3
params.YoungAgeTreshold = 15
params.SpeciesMaxStagnation = 100
params.OldAgeTreshold = 35
params.MinSpecies = 3
params.MaxSpecies = 10
params.RouletteWheelSelection = True
params.RecurrentProb = 0.2
params.OverallMutationRate = 0.02
params.MutateWeightsProb = 0.90
params.WeightMutationMaxPower = 1.0
params.WeightReplacementMaxPower = 5.0
params.MutateWeightsSevereProb = 0.5
params.WeightMutationRate = 0.75
params.MaxWeight = 8
params.MutateAddNeuronProb = 0.01
params.MutateAddLinkProb = 0.02
params.MutateRemLinkProb = 0.00

params.Elitism = 0.1
params.CrossoverRate = 0.5
params.MutateWeightsSevereProb = 0.01

params.MutateNeuronTraitsProb = 0
params.MutateLinkTraitsProb = 0

rng = NEAT.RNG()
rng.TimeSeed()

env = gym.make('FastsimSimpleNavigation-v0')
env.reset()




import random as rnd
import sys
import pickle


def main():

    g = NEAT.Genome(0, 5, 3, 2, False,
                    NEAT.ActivationFunction.LINEAR, NEAT.ActivationFunction.LINEAR, 0, params, 0)
    
    pop = NEAT.Population(g, params, True, 1.0, rnd.randint(0, 1000))

    print('NumLinks:', g.NumLinks())
    #sys.exit(0)

    best_genome_ever = None
    best_ever = -np.inf
    fast_mode = True
    evhist = []
    best_gs = []
    hof = []

    if not ns_on:

        # rtNEAT mode
        print('============================================================')
        print("Please wait for the initial evaluation to complete.")
        fitnesses = []
        
        for _, genome in enumerate(NEAT.GetGenomeList(pop)):
            fitness,behaviour,logs = eval_nn(genome, **kw)
            print('Evaluating',_, " : ",fitness)
            fitnesses.append(fitness)
        for genome, fitness in zip(NEAT.GetGenomeList(pop), fitnesses):
            genome.SetFitness(fitness)
            genome.SetEvaluated()
        maxf = max([x.GetFitness() for x in NEAT.GetGenomeList(pop)])
        print("Max f au debut : ", maxf)
        print('======================')
        print('rtNEAT phase')
        pb = pbar.ProgressBar(maxval=max_evaluations)
        pb.start()
        for i in range(max_evaluations):
            # get best fitness in population and print it
            fitness_list = [x.GetFitness() for x in NEAT.GetGenomeList(pop)]
            best = max(fitness_list)
            #print("Fitness list : ", fitness_list)
            #print("Max  : ", best)
            evhist.append(best)
            
            if best > best_ever:
                sys.stdout.flush()
                print()
                print('NEW RECORD!')
                print('Evaluations:', i, 'Species:', len(pop.Species), 'Fitness:',10000- best)
                best_gs.append(pop.GetBestGenome())
                best_ever = best
                hof.append(pickle.dumps(pop.GetBestGenome()))

            # get the new baby
            old = NEAT.Genome()
            baby = pop.Tick(old)

            # evaluate it
            f,bh,_ = eval_nn(baby,**kw)
            #print("fitness baby : ", f)
            baby.SetFitness(f)
            baby.SetEvaluated()
            fitness_list = [x.GetFitness() for x in NEAT.GetGenomeList(pop)]
            #print("Fitness list at end of step : ", fitness_list)

            pb.update(i)
            sys.stdout.flush()
    else:

        archive = []

        # novelty search
        print('============================================================')
        print("Please wait for the initial evaluation to complete.")
        fitnesses = []
        for _, genome in enumerate(NEAT.GetGenomeList(pop)):
            print('Evaluating',_)
            fitness, behavior, _ = eval_nn(genome, **kw)
            # associate the behavior with the genome
            genome.behavior = behavior

        # recompute sparseness
        def sparseness(genome):
            distances = []
            for g in NEAT.GetGenomeList(pop):
                d = genome.behavior.distance_to( g.behavior )
                distances.append(d)
            # get the distances from the archive as well
            for ab in archive:
                distances.append( genome.behavior.distance_to(ab) )
            distances = sorted(distances)
            sp = np.mean(distances[1:ns_K+1])
            return sp

        print('======================')
        print('Novelty Search phase')
        pb = pbar.ProgressBar(maxval=max_evaluations)

        # Novelty Search variables
        evaluations = 0
        evals_since_last_archiving = 0
        quick_add_counter = 0

        # initial fitness assignment
        for _, genome in enumerate(NEAT.GetGenomeList(pop)):
            genome.SetFitness( sparseness(genome) )
            genome.SetEvaluated()

        # the Novelty Search tick
        while evaluations < max_evaluations:
            pb.start()

            global ns_P_min
            evaluations += 1
            pb.update(evaluations)
            sys.stdout.flush()

            # recompute sparseness for each individual
            if evaluations % ns_recompute_sparseness_each == 0:
                for _, genome in enumerate(NEAT.GetGenomeList(pop)):
                    genome.SetFitness( sparseness(genome) )
                    genome.SetEvaluated()

            # tick
            old = NEAT.Genome()
            new = pop.Tick(old)

            # compute the new behavior
            fitness, behavior, log = eval_nn(new, **kw)
            new.behavior = behavior

            # compute sparseness
            sp = sparseness(new)

            # add behavior to archive if above threshold
            evals_since_last_archiving += 1
            if sp > ns_P_min:
                archive.append(new.behavior)
                evals_since_last_archiving = 0
                quick_add_counter += 1
            else:
                quick_add_counter = 0

            if ns_dynamic_Pmin:
                if evals_since_last_archiving > ns_no_archiving_stagnation_threshold:
                    ns_P_min *= ns_Pmin_lowering_multiplier
                    if ns_P_min < ns_Pmin_min:
                        ns_P_min = ns_Pmin_min

                # too much additions one after another?
                if quick_add_counter > ns_quick_archiving_min_evals:
                    ns_P_min *= ns_Pmin_raising_multiplier

            # set the fitness of the new individual
            new.SetFitness(sp)
            new.SetEvaluated()

            # still use the objective search's fitness to know which genome is best
            if fitness > best_ever:
                sys.stdout.flush()
                print()
                print('Nouveau record MAMEN !')
                print('Evaluations:', evaluations, 'Species:', len(pop.Species), 'Fitness:',10000-fitness)
                hof.append(pickle.dumps(new))
                best_ever = fitness

        pb.finish()

    # Show the best genome's performance forever
    while True:
        hg = pickle.loads(hof[-1])
        evaluate(hg)


In [26]:
main()

NumLinks: 10
Please wait for the initial evaluation to complete.
Evaluating 0
Evaluating 1
Evaluating 2
Evaluating 3
Evaluating 4
Evaluating 5
Evaluating 6
Evaluating 7
Evaluating 8
Evaluating 9
Evaluating 10
Evaluating 11
Evaluating 12
Evaluating 13
Evaluating 14
Evaluating 15
Evaluating 16
Evaluating 17
Evaluating 18
Evaluating 19
Evaluating 20
Evaluating 21
Evaluating 22
Evaluating 23
Evaluating 24
Evaluating 25
Evaluating 26
Evaluating 27
Evaluating 28
Evaluating 29
Evaluating 30
Evaluating 31
Evaluating 32
Evaluating 33
Evaluating 34
Evaluating 35
Evaluating 36
Evaluating 37
Evaluating 38
Evaluating 39
Evaluating 40
Evaluating 41
Evaluating 42
Evaluating 43
Evaluating 44
Evaluating 45
Evaluating 46
Evaluating 47
Evaluating 48
Evaluating 49
Evaluating 50
Evaluating 51
Evaluating 52
Evaluating 53
Evaluating 54
Evaluating 55
Evaluating 56
Evaluating 57
Evaluating 58
Evaluating 59
Evaluating 60
Evaluating 61
Evaluating 62
Evaluating 63
Evaluating 64
Evaluating 65
Evaluating 66
Evaluat

Evaluating 198
Evaluating 199
Novelty Search phase



Nouveau record MAMEN !
Evaluations: 1 Species: 1 Fitness: 376.56938755161536



Nouveau record MAMEN !
Evaluations: 3 Species: 1 Fitness: 267.9457634924911



Nouveau record MAMEN !
Evaluations: 15 Species: 1 Fitness: 250.53498372558533


...


Nouveau record MAMEN !
Evaluations: 7718 Species: 9 Fitness: 40.91663580447312



Nouveau record MAMEN !
Evaluations: 8238 Species: 9 Fitness: 40.10566339110119


Target REACHED ! 



Nouveau record MAMEN !
Evaluations: 11336 Species: 10 Fitness: 9.45560241410385


KeyboardInterrupt: 

In [57]:
help(NEAT.Neuron)

Help on class Neuron in module MultiNEAT._MultiNEAT:

class Neuron(Boost.Python.instance)
 |  Method resolution order:
 |      Neuron
 |      Boost.Python.instance
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__( (object)arg1) -> None :
 |      
 |          C++ signature :
 |              void __init__(_object*)
 |  
 |  __reduce__ = <unnamed Boost.Python function>(...)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  a
 |  
 |  activation
 |  
 |  activation_function_type
 |  
 |  b
 |  
 |  bias
 |  
 |  split_y
 |  
 |  substrate_coords
 |  
 |  time_const
 |  
 |  type
 |  
 |  x
 |  
 |  y
 |  
 |  z
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __instance_size__ = 192
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from B